# Demo for CLIFuzz

## What is CLIFuzz?

`CLIFuzz` is a tool to fuzz command-line interface (CLI) utilities using their configuration options and arguments. It first automatically extracts the configuration options and arguments from CLI utilities which use `getopt`, `getopt_long` or `getopt_long_only` to parse its CLI invocations. It then constructs a JSON grammar representing its invocation language which is then used to grammar fuzz the utility.

## Prerequisites
We start by installing a few prerequisite Jupyter extensions.

In [ ]:
%pip install jupyter_contrib_nbextensions jupyter_nbextensions_configurator==0.4.1

In [ ]:
import sys

In [ ]:
!{sys.executable} -m jupyter contrib nbextension install --sys-prefix

In [ ]:
!{sys.executable} -m jupyter nbextensions_configurator enable

In [ ]:
!{sys.executable} -m jupyter nbextension enable toc2/main

In [ ]:
!{sys.executable} -m jupyter nbextension enable collapsible_headings/main

Please Restart and refresh the Jupyter notebook at this point.

## Walk Through

### Step 1. 
We start with a UNIX command line.

### Step 2.
Consider an example CLI utility `cat` (v9.0) on the Linux command line that uses `getopt_long` to parse its invocations.

In [ ]:
!which cat

### Step 3.


In [ ]:
!cat --help

We observe that `cat` can have multiple options in its invocation e.g., `-A`, `--show-ends` etc.

### Step 4.
Fuzzing `cat`

In [ ]:
import string, random
def fuzzer(max_length, choices):
    string_length = random.randrange(1, max_length + 1)
    return ''.join([random.choice(choices)
                    for c in range(string_length)])

In [ ]:
chars = [c for c in string.printable if c not in "\n\t"]
for i in range(100):
    s = fuzzer(100, chars)
    print('%s %s' % ('cat', s))

### Step 5.
When we fuzz `cat` without incorporating its options in the fuzzing invocation, we miss out on a lot of code segments which can only be triggered by these options. A trivial example in `cat` is the code parsing its invocation:

    while ((c = getopt_long (argc, argv, "benstuvAET", long_options, NULL)) != -1) {
        switch (c) {
            ...
            case 'E':
                show_ends = true;
                break;
            ...
        }

The configuration options activate some flags/variables in this code, e.g., setting the `E` option in `cat` sets the `show_ends` flag to `true`. This flag in turn activates some code segment in the code. E.g., the value of the `show_ends` variable dictates the execution of the following code segment, among others, in `cat`. Excluding the `-E` option from the fuzzing invocations will never execute such code segments.

    if (show_ends)
      {
        if (pending_cr)
          {
            *bpout++ = '^';
            *bpout++ = 'M';
            pending_cr = false;
          }
        *bpout++ = '$';
      }      

### Step 6.
The table below lists the difference in code coverage between fuzzing `cat` without options using `AFL++` and fuzzing `cat` with options using `CLIFuzz`. We see that `CLIFuzz` is able to achieve better Line and Branch Coverage.

| Utility |#lines |#branches| AFL++ Line Coverage | AFL++ Branch Coverage | CLIFuzz Line Coverage | CLIFuzz Branch Coverage |
|---|---|---|---|---|---|---|
| `cat` (v9.0) |1044|784|33.94|22.71|35.81|25.56| 

### Step 7.
How CLIFuzz works (slides)

### Step 8.
The script `run-cf-cmd.py` is the main script. It extracts the options and generates grammars of utilities. It also runs the fuzzer on the utilities using their grammars. Its `help` output describes its usage -

 !run-cf-cmd.py --help

### Step 9
By invoking `run-cf-cmd.py` on `cat`, we can extract its options and the resultant grammar.


In [ ]:
!run-cf-cmd.py  --get-grammar -o cat-gram.json ./latest-targets/coreutils/src/ls

### Step 10.

We show how to produce inputs from this grammar and show the resulting command-line invocations

### Step 11. 
We fuzz cat with CLIFuzz

In [ ]:
!python3 run-cf-cmd.py \
   -f 1000 -g grammar/ls-gram.json \
   -o outputdir/ls.out \
   --log-pass ./latest-targets/coreutils/src/ls

### Step 12.
We show the increased coverage

### Step13.
Conclusion.